In [3]:
!pip install httpx loguru pandas

In [4]:
!pip install httpx loguru pandas

In [5]:
!pip install httpx[http2] loguru pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.3 MB/s eta 0:00:00


In [6]:
!pip install httpx[http2] loguru pandas nest_asyncio

In [7]:
import nest_asyncio
import asyncio
import json
import pandas as pd
from typing import List, Dict
from httpx import AsyncClient, Response
from loguru import logger as log

# Aplicar el parche de nest_asyncio
nest_asyncio.apply()

# Inicializar un cliente httpx asíncrono
client = AsyncClient(
    http2=True,
    headers={
        "Accept-Language": "en-US,en;q=0.9",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Cookie": "intl_splash=false"
    },
)


def parse_reviews(response: Response) -> Dict:
    """Analizar los datos de las reseñas de las respuestas de la API de reseñas"""
    data = response.json()
    total_pages = data["totalPages"]
    review_data = data["topics"]
    return {"data": review_data, "total_pages": total_pages}

async def scrape_reviews(skuid: int) -> List[Dict]:
    """Obtener datos de reseñas desde la API de reseñas"""
    first_page = await client.get(f"https://www.bestbuy.com/ugc/v2/reviews?page=1&pageSize=20&sku={skuid}&sort=MOST_RECENT")
    data = parse_reviews(first_page)
    review_data = data["data"]
    total_pages = data["total_pages"]

    log.info(f"Obteniendo paginación de reseñas, {total_pages - 1} páginas más")
    to_scrape = [
        client.get(f"https://www.bestbuy.com/ugc/v2/reviews?page={page_number}&pageSize=20&sku={skuid}&sort=MOST_RECENT")
        for page_number in range(2, total_pages + 1)
    ]
    for response in asyncio.as_completed(to_scrape):
        response = await response
        data = parse_reviews(response)["data"]
        review_data.extend(data)

    log.success(f"Obtenidas {len(review_data)} reseñas de la API de reseñas")
    return review_data

async def run():
    skuid = 6569668  # ID de SKU del producto
    review_data = await scrape_reviews(skuid)

    # Crear un DataFrame de pandas con los datos de las reseñas
    df = pd.DataFrame(review_data)

    # Mostrar el DataFrame
    print(df.head())

    # Almacenar el DataFrame en un archivo JSON
    df.to_json("reviews.json", orient="records", lines=True)

    return df

# Ejecutar la función run
df = asyncio.get_event_loop().run_until_complete(run())

2024-09-17 14:56:58.017 | INFO     | __main__:scrape_reviews:39 - Obteniendo paginación de reseñas, 10 páginas más
2024-09-17 14:56:58.415 | SUCCESS  | __main__:scrape_reviews:49 - Obtenidas 216 reseñas de la API de reseñas


                                     id topicType  rating  recommended  \
0  766e8ed0-a4a3-35e1-9cb7-ae6f5081b9c8    review       5         True   
1  21982890-7761-3eed-917b-11087412cbfa    review       5         True   
2  428df55d-d6f9-399a-9e54-d5834140c123    review       5         True   
3  20d8d8ed-6126-38af-a1e4-12e6a63cf42f    review       5         True   
4  7fbf75fb-8110-36fa-b1d6-10723dc29655    review       5         True   

                          title  \
0            Excellent for work   
1        Good everyday notebook   
2        Touchscreen Excellence   
3  Best Buy of the day, for me!   
4              Great for school   

                                                text    author  \
0  Does what it’s supposed to do. Fast enough to ...  Akromius   
1  I use to browse internet, show online and make...      Dean   
2  Love it! Best Buy Y must I put 50 characters t...  Khanicsd   
3  So happy I was able to get this laptop. Ordere...    PaulaH   
4  Works great

In [ ]:
from google.colab import files
files.download("reviews.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=b10877d6493a071ea8b28b8b54a809f6adf0796a4501c170e0f537358c75dc0a
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Reviews Transformation") \
    .getOrCreate()

# Leer el archivo JSON
df = spark.read.json("reviews.json")

# Seleccionar y renombrar las columnas necesarias
transformed_df = df.select(
    col("id").alias("id"),
    col("text").alias("text"),
    col("rating").alias("rating")
)

# Mostrar el DataFrame transformado
transformed_df.show()

# Guardar el DataFrame en un archivo CSV
transformed_df.write.csv("reviews_transform.csv", header=True)

# Parar la sesión de Spark
spark.stop()


+--------------------+--------------------+------+
|                  id|                text|rating|
+--------------------+--------------------+------+
|766e8ed0-a4a3-35e...|Does what it’s su...|     5|
|21982890-7761-3ee...|I use to browse i...|     5|
|428df55d-d6f9-399...|Love it! Best Buy...|     5|
|20d8d8ed-6126-38a...|So happy I was ab...|     5|
|7fbf75fb-8110-36f...|Works great!  It ...|     5|
|cb93d5a0-c2ce-303...|It has been a ver...|     5|
|7bda1e1f-76fb-369...|Pretty good for t...|     2|
|8cbdd489-64fe-3e2...|The computer does...|     5|
|2d1b08a7-39bc-318...|I am quite please...|     4|
|9e8858ba-a538-3ef...|Bought this to re...|     5|
|28451835-08af-35e...|My kid loves it ,...|     5|
|4f053de3-5898-366...|Just what I wante...|     5|
|e6d543a5-e643-3c0...|very good custome...|     5|
|def65021-83f3-341...|I Love this Lapto...|     5|
|770510af-bb38-305...|I LOVE this lapto...|     5|
|ec892293-6be9-326...|This laptop is pe...|     5|
|5f3a8a11-3d3a-3e9...|Great lap

In [10]:
import os

# Verificar que el archivo CSV se haya creado
csv_files = [f for f in os.listdir() if f.startswith("reviews_transform")]
print(csv_files)


['reviews_transform.csv']


In [11]:
import shutil

# Empaquetar los archivos CSV en un solo archivo ZIP para facilitar la descarga
shutil.make_archive("reviews_transform", 'zip', '.', 'reviews_transform.csv')

# Descargar el archivo ZIP
from google.colab import files
files.download("reviews_transform.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>